In [3]:
from pathlib import Path
import shutil
import random
import os
from fastprogress import progress_bar
from scipy.ndimage.interpolation import zoom as npzoom
from fastprogress import progress_bar, master_bar
import PIL.Image
import numpy as np
import czifile

relative_path = os.path.abspath(os.path.join('..'))
if relative_path not in sys.path:
    sys.path.append(relative_path)
from utils import get_czi_shape_info, build_index
from skimage.util import random_noise, img_as_ubyte
from skimage import filters
from pdb import set_trace

In [4]:
data_path = Path('/home/bpho/Dropbox (BPHO)')
sources = data_path/'bpho_DL_datasources'

dname = 'transfer_learning_neuron_002'
dpath = data_path/'datasets'/dname
if dpath.exists(): shutil.rmtree(dpath)

hr_path = dpath/'hr'
lr_path = dpath/'lr'
lr_up_path = dpath/'lr_up'

valid_pct = 0.2

In [15]:
# unpaired images (mito, fromserver, neuron, microtubule, moreNeuron)
# neuron_1 movies
neuron_1 = sources/'Live neuron imaging for deep learning longer acquisitions'
hr_neuron_1 = list(neuron_1.glob('*-0*.czi'))

neuron_1_train = [fn for fn in hr_neuron_1]

# neuron_2 movies
neuron_2 = sources/'Live Neurons for DL 031219/train'
hr_neuron_2 = list(neuron_2.glob('*.czi'))

# neuron_3 from server
neuron_3 = sources/'live neuron reimaging for DL 04062019'
hr_neuron_3 = list(neuron_3.glob('*high*.czi'))

In [16]:
train_files = []
valid_files = []

In [17]:
def new_crappify(x, scale=4):
    x = random_noise(x, mode='salt', amount=0.005)
    x = random_noise(x, mode='pepper', amount=0.005)
    lvar = filters.gaussian(x, sigma=5)
    x = random_noise(x, mode='localvar', local_vars=lvar*0.5)
    x_down = npzoom(x, 1/scale, order=1)
    x_up = npzoom(x_down, scale, order=1)
    return x_down, x_up

In [18]:
def new_crappify_movie_to_tifs(czi_fn, hr_dir, lr_dir, lr_up_dir, base_name, max_scale=1.05):
    hr_dir, lr_dir, lr_up_dir = Path(hr_dir), Path(lr_dir), Path(lr_up_dir)
    hr_dir.mkdir(parents=True, exist_ok=True)
    lr_dir.mkdir(parents=True, exist_ok=True)
    lr_up_dir.mkdir(parents=True, exist_ok=True)
    with czifile.CziFile(czi_fn) as czi_f:
        proc_axes, proc_shape = get_czi_shape_info(czi_f)
        channels = proc_shape['C']
        depths = proc_shape['Z']
        times = proc_shape['T']
        x,y = proc_shape['X'], proc_shape['Y']
        data = czi_f.asarray()
        for channel in range(channels):
            for depth in range(depths):
                img_max = None
                for time_col in range(times):
                    try:
                        save_fn = f'{base_name}_{channel:02d}_{depth:03d}_{time_col:03d}.tif'
                        idx = build_index(proc_axes, {'T': time_col, 'C': channel, 'Z':depth, 'X':slice(0,x),'Y':slice(0,y)})
                        img = data[idx].astype(np.float)
                        img_max = img.max() * max_scale
                        if img_max==0: continue #do not save images with no contents.
                        img /= img_max
                        down_img, down_up_img = new_crappify(img)
                    except:
                        continue

                    img = img_as_ubyte(img)
                    pimg = PIL.Image.fromarray(img, mode='L')
                    cur_size = pimg.size
                    pimg.save(hr_dir/save_fn)

                    small_img = PIL.Image.fromarray(img_as_ubyte(down_img))
                    big_img = PIL.Image.fromarray(img_as_ubyte(down_up_img))
                    small_img.save(lr_dir/save_fn)
                    big_img.save(lr_up_dir/save_fn)

In [20]:
for lst in [hr_neuron_1, hr_neuron_2, hr_neuron_3]:
    lst.sort()
    random.shuffle(lst)
    split_idx = int(valid_pct * len(lst))
    valid_files += lst[-split_idx:]
    train_files += lst[:-split_idx]

    for subdir, file_list in [('train', train_files),('valid', valid_files)]:
        print(f'\n\ncopy, crappify and upsample {subdir} files\n\n')
        for fn in progress_bar(file_list):
            base_name = fn.stem
            new_crappify_movie_to_tifs(
                fn, hr_path/subdir, lr_path/subdir, lr_up_path/subdir,
                base_name, max_scale=1.05)
print('done')



copy, crappify and upsample train files




/home/bpho/.conda/envs/fastai/lib/python3.7/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))




copy, crappify and upsample valid files






copy, crappify and upsample train files






copy, crappify and upsample valid files






copy, crappify and upsample train files






copy, crappify and upsample valid files




done
